# Fast-Trips Tutorial 1: Run Fast-Trips for a Single Trip on Simple Network
## Install required software and run this notebook from a virtual environment

The easiest way to get this notebook to work locally is to [download Anaconda, for Python 2.7 ](https://www.continuum.io/downloads)

You can install all the dependencies by importing the virtual environment `ft_tutorial.yml` that is included this git repository.

To run each cell, highlight the cell and use shift-enter.

In [ ]:
import os,datetime
import pandas as pd

# Specify Input Networks + Visualize
Where are the networks?  Please update `BASE_DIR` to reflect where you installed the tutorial on your machine

In [ ]:
BASE_DIR         = r"PUT YOUR PATH TO THE TUTORIAL DIRECTORY HERE!"

INPUT_NETWORKS   = os.path.join(BASE_DIR, r"tta/input/network-simple")
INPUT_DEMAND     = os.path.join(BASE_DIR, r"tta/input/demand-single")

## Validate GTFS Feed
The input networks are stored in the [GTFS-Plus](https://github.com/osplanning-data-standards/GTFS-PLUS) format.  

We use [`transitfeed`](https://github.com/google/transitfeed) for parsing and validating the GTFS component of GTFS plus.  

Note that `transitfeed` does not expect all the extra files associated with GTFS-PLUS and it will be noisy about them.

In [ ]:
import transitfeed
loader        = transitfeed.Loader(INPUT_NETWORKS, memory_db=True)
schedule      = loader.Load()
schedule.Validate()

## Create Simple Map of Routes

We use [`folium`](https://github.com/python-visualization/folium) to create a simple vizualization of the input network.  

The very simple code to create this map is stored in the file `tutorial_map.py` which is included in this repository.  You can make it fancier if you want to see more things!

Note that this would be very slow if you wanted to look at a large city's transit network, but it works for our simple example.  

Folium is a python wrapper around the popular [`Leaflet.js`](http://leafletjs.com/) mapping package.

In [ ]:
import folium
import tutorial_map

mymap = tutorial_map.make_map(schedule)
mymap

# Run Fast-Trips Example: Single Trip
Here we run a simple example of a single rider going from approximately Reynolds Coliseium to the Pullen Park train.

In [ ]:
from fasttrips import Run

### Parameters and Run Configurations
Let's look at the parameter files and run configurations.

In [ ]:
INPUT_WEIGHTS    = os.path.join(BASE_DIR,"tta","input","demand-single","pathweight_ft.txt")
RUN_CONFIG       = os.path.join(BASE_DIR, r"tta","input","demand-single","config_ft.txt")

**Parameters File:**  `pathweights_ft.txt`

In [ ]:
pathweights_df = pd.read_csv(INPUT_WEIGHTS)
pathweights_df

**Run Config File:**  `config_ft.txt`

In [ ]:
with open(RUN_CONFIG, 'r') as myfile:
    contents=myfile.read()
print contents

Fast-Trips can be run with the Run.run_fasttrips command which is a wrapper function that runs:
  * `ft = Run.run_setup()`
  * `ft.run_setup()`
  * `ft.runassignment()`

There are six required inputs:

  * `input_network_dir`
  * `input_denand_dir`
  * `run_config`
  * `input_weights`
  * `output_dir`
  * `pathfinding_type`
  * `iters`

In [ ]:
print Run.run_fasttrips.__doc__

In [ ]:
print Run.run_setup.__doc__

In [ ]:
OUTPUT_DIR    = os.path.join(BASE_DIR,"tta","output")
OUTPUT_FOLDER = "test_simplenet_noOverlap"
Run.run_fasttrips(input_network_dir= INPUT_NETWORKS,
                  input_demand_dir = INPUT_DEMAND,
                  run_config       = RUN_CONFIG,
                  input_weights    = INPUT_WEIGHTS,
                  output_dir       = OUTPUT_DIR,
                  iters            = 1,
                  output_folder    = OUTPUT_FOLDER,
                  dispersion       = 0.5)

# Examine Results

## Vehicle Trip Level
The `veh_trips.csv` file is a record of every stop on every transit trip in the network.  It contains information about arrivals and departures as well as ridership totals.  It is a useful file for examining capacity and operational issues and can be fed back into a dynamic traffic assignment procedure.

In [ ]:
full_output_directory=os.path.join(OUTPUT_DIR,OUTPUT_FOLDER)
vehicles_df = pd.read_csv(os.path.join(full_output_directory,"veh_trips.csv"), 
                                       sep=",", 
                                       parse_dates=['arrival_time', 'departure_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )
vehicles_with_boardings_df=vehicles_df[vehicles_df["onboard"]>0]
vehicles_with_boardings_df[['iteration','pathfinding_iteration','arrival_time','departure_time','route_id','trip_id','stop_id','boards','onboard','standees']]

## Plot Ridership by Trip
As you can see from examining the vehicle trips file, there were two pathfinding iterations, which will be discussed in a later tutorial.  Let's look at the results from the following one, `pathfinding_iteration==1`. 

The resulting plot is not very interesting, since we just sent one trip through, but you can see that they took Trip B2 followed by G3.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=1,figsize=(8,4))
sns.barplot(ax=axs, x="trip_id", 
            order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],
            y="boards", hue="route_id",  
            estimator=sum, 
            ci=None, 
            data=vehicles_with_boardings_df[vehicles_with_boardings_df["pathfinding_iteration"]==1])
axs.set_title('Ridership by Trip', fontsize=24,color="Gray")

## Person Level

The person-level results use the [`dyno-path`](https://github.com/osplanning-data-standards/dyno-path) format.  

There are two levels of files: *path* and *link*.  

Additionally, we store both the *chosen* path and links as well as all the paths and links that were available choices within Fast-Trips. 



### Links on Chosen Path
Links represent a segment of the path using a single transit `trip_id` or access, egress, or transfer linkmode.  


We use [`Bokeh`](http://bokeh.pydata.org/) to examine the results.  Bokeh is a python wrapper on the popular [`D3.js`](https://d3js.org/).  

**Tip**: Sometimes Bokeh is ornery and doesn't play nice.  Usually re-importing it using command below will do the trick.

In [ ]:
from bokeh.charts import Bar, Histogram, TimeSeries, output_file, Line
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

In [ ]:
CHOSEN_LINKS = "chosenpaths_links.csv"
chosen_link_df = pd.read_csv(os.path.join(full_output_directory,CHOSEN_LINKS), 
                                       sep=",", 
                                       parse_dates=['new_A_time', 'new_B_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

chosen_link_df

In [ ]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid,DatetimeAxis, LabelSet
from bokeh.models.glyphs import Segment
from bokeh.io import curdoc, show

mode2color = {'local_bus':"#b3de69",
             'walk_access':'#8dd3c7',
             'walk_egress':'#bebada',
             'transfer':'#fb8072',
             'other':'#80b1d3'}

def assignColorByMode(mode):
    if mode in mode2color.keys():
        return mode2color[mode]
    return mode2color["other"]
    
chosen_link_df["color"]= map(assignColorByMode, chosen_link_df["mode"])

def createAnnotation(mode,sim_cost,route_id,trip_id):
    return str(mode)+" "+str(sim_cost)+" "+str(route_id)+":"+str(trip_id)

chosen_link_df["annotation"]= map(createAnnotation, chosen_link_df["mode"],chosen_link_df["sim_cost"],chosen_link_df["route_id"],chosen_link_df["trip_id"])

source = ColumnDataSource(chosen_link_df)

In [ ]:
xdr = DataRange1d()
ydr = DataRange1d()

plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=800, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(y0="linknum", x0="new_A_time", y1="linknum", x1="new_B_time", line_color="color", line_width="sim_cost")
plot.add_glyph(source, glyph)

xaxis = DatetimeAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

mode_labels = LabelSet(x="new_A_time", y="linknum", text="mode", y_offset=-5,x_offset=10,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='left')
plot.add_layout(mode_labels)

curdoc().add_root(plot)

show(plot, notebook_handle=True)

The plot above shows the chosen path over time with the thicknesses representing the perceived cost of each segment.

## Path Choice

### Path Set
**Path** files contain information about the entire path as scored by the individual who is taking a particular trip.

The fields `sim_cost` and `probability` summarize how the path was scored.    

A quick description of the path is available in the `description` field.

Fast-Trips outputs two path files: `chosenpaths_paths.csv` and `pathset_paths.csv`.  Since our simple example only has a few paths, let's see how each of them were evaluated.

In [ ]:
PATHS  = r"pathset_paths.csv"

pathset_paths_df = pd.read_csv(os.path.join(full_output_directory,PATHS), sep=",")
pd.set_option('display.max_colwidth',160) #widen so you can see whole description
pathset_paths_df[["person_id","person_trip_id","pathnum","description","sim_cost","logsum","probability"]]

### Choice Set Link Files
We can look at the components of each of these paths to find out what is driving the total cost by using the **pathset-links** file: `pathset_links.csv`.

In [ ]:
LINKS  = r"pathset_links.csv"
linkfile_df = pd.read_csv(os.path.join(full_output_directory,LINKS),
                                       sep=",", 
                                       parse_dates=['new_A_time', 'new_B_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

## Add Fast-Trips path probability information to Fast-Trips link file
linkfile_df = pd.merge(left  = linkfile_df,
                       right = pathset_paths_df,
                       how   = "left",
                       left_on = ["person_id","person_trip_id","pathnum","chosen","missed_xfer"],
                       right_on = ["person_id","person_trip_id","pathnum","chosen","missed_xfer"])

#linkfile_df.head()
#linkfile_df.columns

In [ ]:

maxlinknum = linkfile_df["linknum"].max()
    
linkfile_df["color"]= map(assignColorByMode, linkfile_df["mode"])

def createAnnotation(mode,sim_cost,route_id,trip_id,probability):
    if mode not in ['local_bus']:
        return str(mode)+" "+str(sim_cost)
    else:
        return str(mode)+" "+str(sim_cost)+" "+str(route_id)+":"+str(trip_id)

linkfile_df["annotation"]= map(createAnnotation, linkfile_df["mode"],linkfile_df["sim_cost_x"],linkfile_df["route_id"],linkfile_df["trip_id"],linkfile_df["probability"])

def yloc(pathnum,linknum):
    return (pathnum*(maxlinknum+1))+linknum

linkfile_df["yloc"]= map(yloc, linkfile_df["pathnum"],linkfile_df["linknum"])

linksource = ColumnDataSource(linkfile_df)
#linkfile_df

In [ ]:
xdr2 = DataRange1d()
ydr2 = DataRange1d()

choiceplot = Plot(
    title=None, x_range=xdr2, y_range=ydr2, plot_width=800, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = Segment(y0="yloc", x0="new_A_time", y1="yloc", x1="new_B_time", line_color="color", line_width="sim_cost_x")
choiceplot.add_glyph(linksource, glyph)

choicexaxis = DatetimeAxis()
choiceplot.add_layout(choicexaxis, 'below')

choiceplot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
choiceplot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

choice_labels = LabelSet(x="new_A_time", y="yloc", text="annotation", y_offset=-5,x_offset=10,
                  text_font_size="8pt", text_color="#555555",
                  source=linksource, text_align='left')
choiceplot.add_layout(choice_labels)

curdoc().add_root(choiceplot)

##TODO add nodes
##TODO standardize colors by mode
##TODO add probability
##TODO make each path start at a grid line so you can evaluate cost easier

show(choiceplot, notebook_handle=True)

## QUESTION
**Question 1a:** What is the best route according to Tuffie?

**Question 1b:** What drives it being the best?